<a href="https://colab.research.google.com/github/tjrgh822/class_itsystem/blob/master/rotate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Dropout, Flatten, Dense, Input

from numpy import expand_dims
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

train, test= mnist.load_data()
train_data, train_labels, test_data, test_labels = train[0], train[1], test[0], test[1]

In [0]:
from scipy import ndimage, misc
from PIL import Image
data = img_to_array(train_data)
rotate_class = range(0, 90, 5)


data = data.reshape(-1, 28, 28, 1)
random_angle = np.random.choice(rotate_class,  train_data.shape[0]).astype(np.float32)

rotate_data = np.zeros((1, 28, 28, 1), dtype=np.float32)
for i in range(data.shape[0]):
  tmp = Image.fromarray(np.squeeze(ndimage.rotate(data[i], random_angle[i])))
  rotate_data = np.append(rotate_data, tmp)
rotate_data = np.delete(rotate_data, 0, axis=0)

rotate_data.shape


#rotate_data = rotate_data.reshape(-1, 34, 34, 1)

'''rotate_labels = random_angle

for i in np.random.randint(0, 1000, 10):
  plt.figure()
  plt.imshow(rotate_data[i])'''



In [0]:
import cv2
import imutils
data = train_data
rotate_class = range(0, 90, 5)


data = data.reshape(-1, 28, 28)
random_angle = np.random.choice(rotate_class,  train_data.shape[0])

a = imutils.rotate(data[i], random_angle[i])

print(a.dtype)

rotate_data = np.zeros(1, dtype=np.uint8)
for i in range(len(data)):
  rotate_data = np.append(rotate_data, np.array(imutils.rotate(data[i], random_angle[i])))
  if i % 10000 == 0:
    print(i)

rotate_data = np.delete(rotate_data, 0).reshape(-1, 28, 28, 1)


In [0]:
np.save('rotate_data.npy', rotate_data)

In [0]:
data = img_to_array(test_data)
rotate_class = range(0, 90, 5)


data = data.reshape(-1, 28, 28, 1)
random_angle = np.random.choice(rotate_class,  train_data.shape[0]).astype(np.float32)
test_rotate_data = np.array([np.squeeze(ndimage.rotate(data[i], random_angle[i])) for i in range(data.shape[0])])

test_rotate_data = test_rotate_data.resahpe(-1, 34, 34, 1)

test_rotate_labels = random_angle

In [0]:
np.save("rotate_data.npy", rotate_data)

In [0]:
np.save("rotate_labels.npy", rotate_labels)

In [0]:
def angle_difference(x, y):
    """
    Calculate minimum difference between two angles.
    """
    return 180 - abs(abs(x - y) - 180)


def angle_error(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a binary vector.
    """
    diff = angle_difference(K.argmax(y_true), K.argmax(y_pred))
    return K.mean(K.cast(K.abs(diff), K.floatx()))


def angle_error_regression(y_true, y_pred):
    """
    Calculate the mean diference between the true angles
    and the predicted angles. Each angle is represented
    as a float number between 0 and 1.
    """
    return K.mean(angle_difference(y_true * 360, y_pred * 360))

'''    datagen = ImageDataGenerator( width_shift_range=5, 
                             height_shift_range=5, 
                             horizontal_flip=True, 
                             vertical_flip=True,
                             brightness_range = [0.2, 1.0],
                             zoom_range = [0.5, 1.0]
                             )'''
datagen = ImageDataGenerator()                             

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
nb_filters = 64
pool_size = (2, 2)
kernel_size = (3, 3)
nb_classes = 18

# model definition
inputs = Input(shape=(34, 34, 1))
x = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                  activation='relu')(inputs)
x = Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                  activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(nb_classes, activation='softmax')(x)

model = Model(inputs, x)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[angle_error])

model.summary()







In [0]:
batch = 100
epochs = 10

# callbacks
checkpointer = ModelCheckpoint(
    filepath='model.h5',
    save_best_only=True
)
early_stopping = EarlyStopping(patience=2)
tensorboard = TensorBoard()

callbacks=[checkpointer, early_stopping, tensorboard]

model.fit(datagen.flow(rotate_data, rotate_labels, batch_size=batch), 
          steps_per_epoch=len(rotate_data) / batch, 
          epochs = epochs, 
          validation_data = datagen.flow(test_rotate_data, test_rotate_labels),
          callbacks= callbacks
          )


In [0]:
from google.colab import drive

drive.mount('/content/gdrive')